# Respondendo perguntas de negócios.

1) Qual dia da semana tem, em média, maior volume de transações e qual tem, também em média,  maior valor movimentado?

2) O BanVic tem, em média, os maiores valores movimentados no início ou final de mês? (Considere início do mês como sendo os primeiro 15 dias e o final do mês sendo os últimos 15 dias de cada mês).

In [3]:
import os
import pandas as pd

from dotenv import load_dotenv

In [4]:
# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Obter caminhos do arquivo csv e do diretório de extração das variáveis de ambiente
transacoes_path = os.getenv('TRANSACOES_CSV')

In [5]:
# Carregar o CSV em um DataFrame Pandas
df_transacoes = pd.read_csv(transacoes_path)

In [6]:
# Converter a coluna 'data_transacao' para datetime
df_transacoes['data_transacao'] = pd.to_datetime(df_transacoes['data_transacao'], format='%Y-%m-%d %H:%M:%S %Z', errors='coerce')

# Remover linhas com datas inválidas
df_transacoes = df_transacoes.dropna(subset=['data_transacao'])

# Extrair o dia da semana e a parte do mês
df_transacoes['dia_semana'] = df_transacoes['data_transacao'].dt.day_name()
df_transacoes['parte_mes'] = df_transacoes['data_transacao'].dt.day.apply(lambda x: 'Início' if x <= 15 else 'Final')

# 1. Qual dia da semana tem, em média, maior volume de transações e maior valor movimentado?
volume_transacoes_por_dia = df_transacoes.groupby('dia_semana')['cod_transacao'].count()
valor_transacoes_por_dia = df_transacoes.groupby('dia_semana')['valor_transacao'].sum()

# Calcular a média de volume e valor
media_volume_por_dia = volume_transacoes_por_dia.mean()
media_valor_por_dia = valor_transacoes_por_dia.mean()

# Encontrar o dia da semana com maior volume e valor movimentado
dia_maior_volume = volume_transacoes_por_dia.idxmax()
dia_maior_valor = valor_transacoes_por_dia.idxmax()

print(f'O dia da semana com maior volume de transações, em média, é: {dia_maior_volume}')
print(f'O dia da semana com maior valor movimentado, em média, é: {dia_maior_valor}')

# 2. O BanVic tem, em média, os maiores valores movimentados no início ou final de mês?
valor_transacoes_por_parte_mes = df_transacoes.groupby('parte_mes')['valor_transacao'].sum()

# Calcular a média de valores movimentados
media_valor_inicio_mes = valor_transacoes_por_parte_mes.loc['Início'] / (df_transacoes['parte_mes'] == 'Início').sum()
media_valor_final_mes = valor_transacoes_por_parte_mes.loc['Final'] / (df_transacoes['parte_mes'] == 'Final').sum()

if media_valor_inicio_mes > media_valor_final_mes:
    maior_valor_periodo = 'Início'
else:
    maior_valor_periodo = 'Final'

print(f'O BanVic tem, em média, os maiores valores movimentados no {maior_valor_periodo} do mês.')

# Resultados com prints:
print(f'Média de volume de transações por dia da semana: {media_volume_por_dia}')
print(f'Média de valor de transações por dia da semana: {media_valor_por_dia}')
print(f'Média de valor movimentado no início do mês: {media_valor_inicio_mes}')
print(f'Média de valor movimentado no final do mês: {media_valor_final_mes}')


O dia da semana com maior volume de transações, em média, é: Thursday
O dia da semana com maior valor movimentado, em média, é: Thursday
O BanVic tem, em média, os maiores valores movimentados no Início do mês.
Média de volume de transações por dia da semana: 8299.57142857143
Média de valor de transações por dia da semana: 3417917.935714286
Média de valor movimentado no início do mês: 440.9489658027656
Média de valor movimentado no final do mês: 391.0893035382848


# Criação da Dimensão de Datas

In [7]:
# Criar um intervalo de datas
start_date = '2010-01-01'
end_date = '2023-12-31'
dates = pd.date_range(start=start_date, end=end_date)

# Criar o DataFrame da dimensão de datas
dim_dates = pd.DataFrame(dates, columns=['date'])

# Adicionar colunas úteis
dim_dates['year'] = dim_dates['date'].dt.year
dim_dates['month'] = dim_dates['date'].dt.month
dim_dates['day'] = dim_dates['date'].dt.day
dim_dates['quarter'] = dim_dates['date'].dt.quarter
dim_dates['semester'] = dim_dates['date'].dt.month.apply(lambda x: 1 if x <= 6 else 2)
dim_dates['day_of_week'] = dim_dates['date'].dt.day_name()
dim_dates['week_of_year'] = dim_dates['date'].dt.isocalendar().week
dim_dates['day_of_year'] = dim_dates['date'].dt.dayofyear
dim_dates['is_weekend'] = dim_dates['day_of_week'].isin(['Saturday', 'Sunday'])

# Visualizar a dimensão de datas
print(dim_dates.head())

        date  year  month  day  quarter  semester day_of_week  week_of_year  \
0 2010-01-01  2010      1    1        1         1      Friday            53   
1 2010-01-02  2010      1    2        1         1    Saturday            53   
2 2010-01-03  2010      1    3        1         1      Sunday            53   
3 2010-01-04  2010      1    4        1         1      Monday             1   
4 2010-01-05  2010      1    5        1         1     Tuesday             1   

   day_of_year  is_weekend  
0            1       False  
1            2        True  
2            3        True  
3            4       False  
4            5       False  


# Novas Análises

**Sazonalidade**

Essa análise pode ajudar a identificar padrões sazonais que possam influenciar a demanda por serviços bancários, permitindo ao banco preparar-se melhor para períodos de alta demanda.

In [8]:
# Agrupar as transações por mês
df_transacoes['year_month'] = df_transacoes['data_transacao'].dt.to_period('M')

# Calcular volume e valor de transações por mês
transacoes_por_mes = df_transacoes.groupby('year_month').agg({'cod_transacao': 'count', 'valor_transacao': 'sum'}).reset_index()
transacoes_por_mes.columns = ['year_month', 'volume_transacoes', 'valor_transacoes']

# Média de volume e valor por mês
media_volume_por_mes = transacoes_por_mes['volume_transacoes'].mean()
media_valor_por_mes = transacoes_por_mes['valor_transacoes'].mean()

print(f'Média de volume de transações por mês: {media_volume_por_mes}')
print(f'Média de valor de transações por mês: {media_valor_por_mes}')

# Identificar os meses com maiores volumes e valores
mes_maior_volume = transacoes_por_mes.loc[transacoes_por_mes['volume_transacoes'].idxmax(), 'year_month']
mes_maior_valor = transacoes_por_mes.loc[transacoes_por_mes['valor_transacoes'].idxmax(), 'year_month']

print(f'O mês com maior volume de transações é: {mes_maior_volume}')
print(f'O mês com maior valor de transações é: {mes_maior_valor}')


Média de volume de transações por mês: 374.81935483870967
Média de valor de transações por mês: 154357.5841935484
O mês com maior volume de transações é: 2022-12
O mês com maior valor de transações é: 2022-12


C:\Users\raull\AppData\Local\Temp\ipykernel_10892\940990071.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_transacoes['year_month'] = df_transacoes['data_transacao'].dt.to_period('M')


**Impacto do Final de Semana**

Como o volume e valor das transações variam entre dias de semana e finais de semana?

Essa análise pode ajudar a entender o comportamento dos clientes em diferentes dias da semana, o que pode ser útil para planejar campanhas de marketing e alocar recursos de atendimento.

In [9]:
# Adicionar coluna indicando se é final de semana
df_transacoes['is_weekend'] = df_transacoes['data_transacao'].dt.dayofweek >= 5

# Calcular volume e valor de transações para dias de semana e finais de semana
transacoes_por_tipo_dia = df_transacoes.groupby('is_weekend').agg({'cod_transacao': 'count', 'valor_transacao': 'sum'}).reset_index()
transacoes_por_tipo_dia.columns = ['is_weekend', 'volume_transacoes', 'valor_transacoes']

# Média de volume e valor por tipo de dia
media_volume_por_tipo_dia = transacoes_por_tipo_dia['volume_transacoes'].mean()
media_valor_por_tipo_dia = transacoes_por_tipo_dia['valor_transacoes'].mean()

print(f'Média de volume de transações por tipo de dia: {media_volume_por_tipo_dia}')
print(f'Média de valor de transações por tipo de dia: {media_valor_por_tipo_dia}')

# Identificar se finais de semana têm maiores volumes e valores
if transacoes_por_tipo_dia.loc[transacoes_por_tipo_dia['is_weekend'], 'volume_transacoes'].values[0] > media_volume_por_tipo_dia:
    maior_volume_dia = 'Finais de Semana'
else:
    maior_volume_dia = 'Dias de Semana'

if transacoes_por_tipo_dia.loc[transacoes_por_tipo_dia['is_weekend'], 'valor_transacoes'].values[0] > media_valor_por_tipo_dia:
    maior_valor_dia = 'Finais de Semana'
else:
    maior_valor_dia = 'Dias de Semana'

print(f'O maior volume de transações ocorre nos: {maior_volume_dia}')
print(f'O maior valor de transações ocorre nos: {maior_valor_dia}')


Média de volume de transações por tipo de dia: 29048.5
Média de valor de transações por tipo de dia: 11962712.775
O maior volume de transações ocorre nos: Dias de Semana
O maior valor de transações ocorre nos: Dias de Semana
